In [1]:
basedir = './imgs/'

In [2]:
import os
import numpy as np
import pandas as pd

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.preprocessing.image import *

np.random.seed(2017)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def img_gen(group, train_gen, valid_gen, basedir, model_image_size, batch_size):
    train_generator = train_gen.flow_from_directory(os.path.join(basedir, 'train'+str(group)),  model_image_size, \
                                                shuffle=True, batch_size=batch_size, class_mode="categorical")

    valid_generator = valid_gen.flow_from_directory(os.path.join(basedir, 'valid'+str(group)),  model_image_size, \
                                          shuffle=True, batch_size=batch_size, class_mode="categorical")
    
    steps_train_sample = int(train_generator.samples)//batch_size + 1
    steps_valid_sample = int(valid_generator.samples)//batch_size + 1
    return train_generator, valid_generator, steps_train_sample, steps_valid_sample 

In [4]:
def build_model(model_name, model_image_size, fune_tune_layer):
    inputs = Input((*model_image_size, 3))
    base_model = model_name(input_tensor=inputs, weights='imagenet', include_top=False)
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(inputs, x)    
    for i in range(fune_tune_layer):
        model.layers[i].trainable = False
    return model

In [5]:
def train_model(model, optimizer, epoch, train_generator, valid_generator, steps_train_sample, steps_valid_sample):
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=epoch, \
                    validation_data=valid_generator, validation_steps=steps_valid_sample)

In [6]:
def predict_model(model, test_generator, steps_test_sample):
    y_pred = model.predict_generator(test_generator, steps=test_generator.samples//batch_size+1, verbose=1)
    return y_pred

In [36]:
def submission(result, fname):
    l = []
    for i, imgname in tqdm(enumerate(test_generator.filenames)):
        name = imgname[imgname.rfind('/')+1:]
        l.append([name, *result[i]])
        results = np.array(l)
    data = {'img': results[:,0]}
    for i in range(10):
        data['c%d'%i] = results[:,i+1]
    df = pd.DataFrame(data, columns=['img'] + ['c%d'%i for i in range(10)])
    df = df.sort_values(by='img')
    df.to_csv(fname, index=None)    

In [39]:
from keras.applications.vgg16 import preprocess_input

In [8]:
from keras.applications.resnet50 import preprocess_input

In [22]:
from keras import backend as K
K.clear_session()

model_image_size = (240, 320)
batch_size = 48
epoch = 1
model_name = ResNet50
fune_tune_layer = 0
optimizer = Adam(lr=1e-4)

train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10.,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.1,
    zoom_range=0.1,
)

gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = gen.flow_from_directory(os.path.join(basedir, 'Test'), target_size=model_image_size, shuffle=False, batch_size=batch_size, class_mode=None)
steps_test_sample = test_generator.samples//batch_size + 1

result = np.array([0.]*79726)
for i in range(2,3):
    train_generator, valid_generator, steps_train_sample, steps_valid_sample = img_gen(i, train_gen, gen, basedir, model_image_size, batch_size)
    model = build_model(model_name, model_image_size, fune_tune_layer)
    train_model(model, optimizer, epoch, train_generator, valid_generator, steps_train_sample//2, steps_valid_sample)
#    optimizer = Adam(lr=1e-5)
#    train_model(model, optimizer, epoch, train_generator, valid_generator, steps_train_sample, steps_valid_sample)
#    optimizer = Adam(lr=1e-6)
#    train_model(model, optimizer, epoch, train_generator, valid_generator, steps_train_sample, steps_valid_sample)
#    result += predict_model(model, test_generator, steps_test_sample)
#result /= 5

#submission(result, 'VGG16.csv')

Found 79726 images belonging to 1 classes.
Found 16684 images belonging to 10 classes.
Found 5740 images belonging to 10 classes.
Epoch 1/1
174/174 [==============================] - 413s 2s/step - loss: 0.4499 - acc: 0.8612 - val_loss: 0.3730 - val_acc: 0.8929


In [23]:
optimizer = Adam(lr=1e-5)
train_model(model, optimizer, epoch, train_generator, valid_generator, steps_train_sample//2, steps_valid_sample)

Epoch 1/1
174/174 [==============================] - 413s 2s/step - loss: 0.0363 - acc: 0.9908 - val_loss: 0.2769 - val_acc: 0.9091


In [ ]:
result = predict_model(model, test_generator, steps_test_sample)

In [42]:
result.shape

(79726, 10)

In [35]:
!pip install tqdm  
from tqdm import tqdm

  Using cached tqdm-4.19.6-py2.py3-none-any.whl


In [45]:
df = pd.read_csv("sample_submission copy.csv")

for i in tqdm(range(result.shape[0])):
    df.iloc[i,1:11] = result[i]
    
df.to_csv('ResNet50.csv', index=None)

100%|██████████| 79726/79726 [05:25<00:00, 244.86it/s]
